In [1]:
import numpy as np

from tqdm import tqdm
import pickle
import pathlib
import torch

In [10]:
index_file = "/home/jovyan/uncompressed/tf_example/training_mapstyle/index_file.txt"
with open(index_file, 'rb') as f:
            index = pickle.load(f)

In [11]:
index

{0: ['training_mapstyle/npz_files/batch_1.npz', 0],
 1: ['training_mapstyle/npz_files/batch_1.npz', 1],
 2: ['training_mapstyle/npz_files/batch_1.npz', 2],
 3: ['training_mapstyle/npz_files/batch_1.npz', 3],
 4: ['training_mapstyle/npz_files/batch_1.npz', 4],
 5: ['training_mapstyle/npz_files/batch_1.npz', 5],
 6: ['training_mapstyle/npz_files/batch_1.npz', 6],
 7: ['training_mapstyle/npz_files/batch_1.npz', 7],
 8: ['training_mapstyle/npz_files/batch_1.npz', 8],
 9: ['training_mapstyle/npz_files/batch_1.npz', 9],
 10: ['training_mapstyle/npz_files/batch_1.npz', 10],
 11: ['training_mapstyle/npz_files/batch_1.npz', 11],
 12: ['training_mapstyle/npz_files/batch_1.npz', 12],
 13: ['training_mapstyle/npz_files/batch_1.npz', 13],
 14: ['training_mapstyle/npz_files/batch_1.npz', 14],
 15: ['training_mapstyle/npz_files/batch_1.npz', 15],
 16: ['training_mapstyle/npz_files/batch_1.npz', 16],
 17: ['training_mapstyle/npz_files/batch_1.npz', 17],
 18: ['training_mapstyle/npz_files/batch_1.npz',

In [7]:
ni = 0
new_index = {}
for key, val in tqdm(index.items()):
    if "batch_0.npz" not in val[0]:
        new_index[ni] = val
        ni+=1

100%|██████████| 437715/437715 [00:00<00:00, 2080756.56it/s]


In [9]:
with open(index_file, "wb") as f:
    pickle.dump(new_index, f)



In [1]:
class WaymoDataset(torch.utils.data.Dataset):
    def __init__(self, ds_path: str, index_file: str):
        self.ds_path = ds_path
        # assert self.ds_path.exists()
        assert pathlib.Path(self.ds_path).exists()
        self.index_file = index_file
        assert pathlib.Path(self.index_file).exists()
        self.index = None
        self.read_files = {}
        with open(index_file, 'rb') as f:
            self.index = pickle.load(f)

    def __getitem__(self, item):
        path = self.index[item][0]
        frame_idx = self.index[item][1]
        # load np from path
        if path not in self.read_files:
            # join ds_path + path
            path_glob = pathlib.Path(self.ds_path) / path
            data = np.load(path_glob, allow_pickle=True)["data"].reshape(-1)[0]
            self.read_files[path] = data
        else:
            data = self.read_files[path]
        chunk = data[frame_idx]
        return chunk

    def __len__(self):
        return len(self.index)

# main
# if __name__ == "__main__":
# ds_path = pathlib.Path("/home/jovyan/uncompressed/tf_example/training_mapstyle")
# assert pathlib.Path(ds_path).exists()

# index_file = "index_file.txt"
# # join
# index_file = pathlib.Path(ds_path) / index_file
# # join the path with the index file
# ds = WaymoDataset(ds_path, index_file)
# print(f"len(ds): {len(ds)}")
# for i in tqdm(range(len(ds))):
#     data = ds[i]
# from scripts.dataloaders import d_collate_fn
# from scripts.rgb_loader import RgbLoader

# rgb_loader = RgbLoader("/home/jovyan/rendered/train/index.pkl")
# loader = torch.utils.data.DataLoader(ds, batch_size=4,
#                                           num_workers=0, collate_fn=d_collate_fn)
# # use tqdm to iterate over the loader
# for i, data in tqdm(enumerate(loader)):
#     rgb = rgb_loader.load_batch_rgb(data, prefix="/home/jovyan/rendered/")

In [15]:
with open("/home/jovyan/uncompressed/tf_example/training_mapstyle/index_file.txt", 'rb') as f:
    index = pickle.load(f)

In [14]:
ls -la  /home/jovyan/uncompressed/tf_example/training_mapstyle/

total 56
drwxr-xr-x 4 jovyan 4294967294  4096 May  5 15:53 ./
drwxr-xr-x 8 jovyan 4294967294  4096 May  5 15:13 ../
drwxr-xr-x 2 jovyan 4294967294  4096 May  5 15:14 .ipynb_checkpoints/
-rw-r--r-- 1 jovyan 4294967294 33254 May  5 15:53 index_file.txt
drwxr-xr-x 3 jovyan 4294967294  4096 May  5 15:53 npz_files/


In [20]:
!ls -l /home/jovyan/rendered/train | grep ^- | wc -l

11940


In [26]:
index_dict = "/home/jovyan/rendered/train/index.pkl"
index_dict2 = "/home/jovyan/waymo-open-dataset/tutorial/rendered/train/index.pkl"

In [23]:
def load_dict( filename_=None):
        
        out_d = {}
        with open(filename_, 'rb') as f:
            d = {}
            try:
                lim = 1e5
                dicts = []
                while 1 and lim > 0:
                    lim -= 1
                    loaded_dict = pickle.load(f)
                    dicts.append(loaded_dict)
                    # out_d = {**out_d, **loaded_dict}
                    
            except EOFError:
                pass
        d = {k:v for e in dicts for (k,v) in e.items()}
        print(f"loaded index file contains {1e5-lim} indexes to files")
        return d

In [24]:
d = load_dict(index_dict)


loaded index file contains 11940.0 indexes to files


In [27]:
d2 = load_dict(index_dict2)

loaded index file contains 6806.0 indexes to files


In [49]:
un = set(d.keys()) ^ set(d2.keys())
inter = set(d.keys()) & set(d2.keys())

In [48]:
print(list(un)[3])
print(list(d2.keys())[8])

ecd18dd5cc184b021855.0
f6f068df2073c857287.0


In [56]:
len(inter)
len(d2)

1039650

In [59]:
"5f439af5d5afa68c474.0" in d.keys()


False

In [75]:
from tqdm import tqdm
for key, val in tqdm(d.items()):
    if "5f43" in key:
        print(key, val)
        # break

 29%|██▉       | 631232/2175544 [00:00<00:00, 3218657.31it/s]

4f04b50b55b8d5f4314.0 rendered/train/rgb221.npz
4f04b50b55b8d5f4324.0 rendered/train/rgb221.npz
a2f1d65f43f84c320.0 rendered/train/rgb295.npz
a2f1d65f43f84c324.0 rendered/train/rgb295.npz
a2f1d65f43f84c3221.0 rendered/train/rgb295.npz
a2f1d65f43f84c3225.0 rendered/train/rgb295.npz
a2f1d65f43f84c3227.0 rendered/train/rgb295.npz
5f439755f3a655ec3.0 rendered/train/rgb309.npz
5f439755f3a655ec7.0 rendered/train/rgb309.npz
5f439755f3a655ec9.0 rendered/train/rgb309.npz
555a30d9be98025f4326.0 rendered/train/rgb498.npz
555a30d9be98025f4333.0 rendered/train/rgb498.npz
555a30d9be98025f4366.0 rendered/train/rgb498.npz
555a30d9be98025f4397.0 rendered/train/rgb498.npz
db207b5f43f100c4376.0 rendered/train/rgb661.npz
db207b5f43f100c4377.0 rendered/train/rgb661.npz
db207b5f43f100c4387.0 rendered/train/rgb661.npz
db207b5f43f100c4715.0 rendered/train/rgb661.npz
e1882a9210e835f43033.0 rendered/train/rgb904.npz
fd189f5f43690b4a2195.0 rendered/train/rgb1054.npz
fd189f5f43690b4a5709.0 rendered/train/rgb1054.

 63%|██████▎   | 1367478/2175544 [00:00<00:00, 3533516.74it/s]

 rendered/train/rgb4978.npz
e696835f43aee6414688.0 rendered/train/rgb4978.npz
e696835f43aee6419198.0 rendered/train/rgb4978.npz
34b6c112faa995f431.0 rendered/train/rgb5045.npz
6e9900e6e1dbb5f43.0 rendered/train/rgb5280.npz
2b6b3535f430c7f0229.0 rendered/train/rgb5330.npz
2b6b3535f430c7f0233.0 rendered/train/rgb5330.npz
2b6b3535f430c7f0235.0 rendered/train/rgb5330.npz
2b6b3535f430c7f02563.0 rendered/train/rgb5330.npz
82d1726445355f432216.0 rendered/train/rgb5338.npz
20b105f43f622dc41.0 rendered/train/rgb5570.npz
20b105f43f622dc431.0 rendered/train/rgb5570.npz
20b105f43f622dc463.0 rendered/train/rgb5570.npz
7985f43c923611995.0 rendered/train/rgb5587.npz
7985f43c9236119917.0 rendered/train/rgb5587.npz
7985f43c9236119922.0 rendered/train/rgb5587.npz
7985f43c9236119923.0 rendered/train/rgb5587.npz
7985f43c9236119924.0 rendered/train/rgb5587.npz
7985f43c9236119926.0 rendered/train/rgb5587.npz
2137066fbbc7125f4322.0 rendered/train/rgb5664.npz
2137066fbbc7125f4324.0 rendered/train/rgb5664.npz


100%|██████████| 2175544/2175544 [00:00<00:00, 3501104.21it/s]

c9fc3ff0ddf4385f43.0 rendered/train/rgb9068.npz
d2a18e95f43336b2997.0 rendered/train/rgb9078.npz
d2a18e95f43336b23335.0 rendered/train/rgb9078.npz
c4f3015f43c5b1591480.0 rendered/train/rgb9138.npz
c4f3015f43c5b1591481.0 rendered/train/rgb9138.npz
c4f3015f43c5b1591496.0 rendered/train/rgb9138.npz
c4f3015f43c5b1591500.0 rendered/train/rgb9138.npz
925f43646b6ad982267.0 rendered/train/rgb9191.npz
925f43646b6ad982279.0 rendered/train/rgb9191.npz
925f43646b6ad9822335.0 rendered/train/rgb9191.npz
925f43646b6ad9822338.0 rendered/train/rgb9191.npz
658c5095f434f73e750.0 rendered/train/rgb9202.npz
658c5095f434f73e1271.0 rendered/train/rgb9202.npz
658c5095f434f73e1273.0 rendered/train/rgb9202.npz
4f5f4303d01e0a8c1447.0 rendered/train/rgb9254.npz
4f5f4303d01e0a8c1449.0 rendered/train/rgb9254.npz
4f5f4303d01e0a8c1455.0 rendered/train/rgb9254.npz
4f5f4303d01e0a8c1457.0 rendered/train/rgb9254.npz
4f5f4303d01e0a8c1460.0 rendered/train/rgb9254.npz
4f5f4303d01e0a8c1535.0 rendered/train/rgb9254.npz
4a7dfb